Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [32]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [33]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)

df = sessionsDF.drop(columns=["timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#interaction matrix is build on part of the sessionDF - not perfect way, because test set is all over the place (random shuffel), but i needed something fast for testing
train, test = train_test_split(df, test_size=0.2, stratify=df['product_id'])

#idea is to make sure that all data in test set has at least two entries from given session. Data that not satisfies that criteria are appended to train set.
s = test['session_id'].value_counts()
res = test[test['session_id'].map(s) >= 2]
ret = test[test['session_id'].map(s) == 1]
test.sort_values(by='session_id', inplace=True)
res.sort_values(by='session_id', inplace=True)
# display(test)
# display(res)

train.append(ret).sort_values(by='session_id', inplace=True)
interactionMatrixDF = pd.pivot_table(train, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
display(interactionMatrixDF)


/home/swirta/.local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,1,4,10,1,3,1,4,2,8,...,0,0,2,1,1,2,6,0,0,7
104,0,1,6,6,5,7,0,4,3,6,...,1,2,1,0,1,1,5,2,6,1
105,0,1,7,7,2,2,0,1,3,3,...,0,1,1,0,2,1,7,4,0,0
106,2,1,2,9,0,3,2,3,1,5,...,0,0,2,3,1,1,4,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2,1,3,11,8,2,0,1,7,4,...,0,1,0,2,0,1,7,1,2,2
298,0,0,6,1,2,2,0,6,1,2,...,1,0,0,0,1,2,8,1,0,0
299,0,0,3,0,1,0,0,1,0,1,...,0,0,1,0,1,1,3,1,0,0


Normalizacja wartości w macierzy interakcji.

In [34]:
interactionMatrixDF = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.00,0.0,0.0625,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.00,0.000,0.00,0.00,0.166667,0.000000,0.000000
1,0.00,0.2,0.2500,0.416667,0.071429,0.230769,0.25,0.363636,0.181818,0.421053,...,0.0,0.000000,0.4,0.25,0.125,0.50,0.24,0.000000,0.000000,1.000000
2,0.00,0.2,0.3750,0.250000,0.357143,0.538462,0.00,0.363636,0.272727,0.315789,...,0.5,0.333333,0.2,0.00,0.125,0.25,0.20,0.333333,1.000000,0.142857
3,0.00,0.2,0.4375,0.291667,0.142857,0.153846,0.00,0.090909,0.272727,0.157895,...,0.0,0.166667,0.2,0.00,0.250,0.25,0.28,0.666667,0.000000,0.000000
4,0.25,0.2,0.1250,0.375000,0.000000,0.230769,0.50,0.272727,0.090909,0.263158,...,0.0,0.000000,0.4,0.75,0.125,0.25,0.16,0.333333,0.000000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.25,0.2,0.1875,0.458333,0.571429,0.153846,0.00,0.090909,0.636364,0.210526,...,0.0,0.166667,0.0,0.50,0.000,0.25,0.28,0.166667,0.333333,0.285714
196,0.00,0.0,0.3750,0.041667,0.142857,0.153846,0.00,0.545455,0.090909,0.105263,...,0.5,0.000000,0.0,0.00,0.125,0.50,0.32,0.166667,0.000000,0.000000
197,0.00,0.0,0.1875,0.000000,0.071429,0.000000,0.00,0.090909,0.000000,0.052632,...,0.0,0.000000,0.2,0.00,0.125,0.25,0.12,0.166667,0.000000,0.000000
198,0.25,1.0,0.5000,0.250000,0.214286,0.384615,0.00,0.454545,0.272727,0.421053,...,0.5,0.166667,0.0,0.00,0.375,1.00,0.28,0.333333,0.000000,0.285714


Utworzenie słownika dla szybszego wyszukiwania.

In [35]:
idxNameDict = pd.Series(productsDF["product_id"].values, index=productsDF.index).to_dict()

Przygotowanie zbirów danych: treningowy i testowy

In [36]:
from sklearn.model_selection import train_test_split

trainUser, testUser = train_test_split(interactionMatrixDF, test_size=0.2)

#transpose because items are columns
# trainItem, testItem = train_test_split(interactionMatrixDF.transpose(), test_size=0.2)
# trainItem = interactionMatrixDF.transpose() #for testing only




Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [37]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

pd.DataFrame(userFeatures)
pd.DataFrame(itemFeatures)


,0,1,2,3,4,5,6,7,8,9
0,1.484534,0.112130,0.031594,-0.452020,-0.293011,0.017635,0.479970,0.462554,0.439099,0.068719
1,2.724852,0.155947,-0.724128,-0.096505,-0.360382,0.257459,-0.395162,0.141047,-0.460759,0.315875
2,3.332543,-0.148504,-0.257223,0.264037,0.066496,-0.359473,-0.147903,0.147361,-0.276975,-0.199857
3,4.285009,-0.489030,0.399519,0.083034,-0.646427,0.237695,-0.161507,-0.367619,-0.171391,0.084938
4,3.671361,0.428669,-0.125027,0.350423,-0.198901,-0.394009,-0.198776,0.122036,-0.112311,-0.376334
...,...,...,...,...,...,...,...,...,...,...
314,2.840791,0.779950,-0.720211,0.174811,0.087015,0.041509,0.225680,0.050781,0.002881,0.121036
315,3.603874,0.340035,-0.538856,-0.231536,0.224415,-0.382791,-0.211682,0.025063,-0.461536,-0.494761
316,3.511144,0.139026,-0.110497,0.343309,0.351775,-0.745756,0.171299,0.218438,-0.230795,-0.557377
317,2.204669,-0.801219,-0.706735,-0.179799,-0.322397,-0.512441,0.235717,0.111467,-0.134302,0.631945


In [38]:
def top_k(item_id, top_k, corr_mat, map_name):
    topItems = corr_mat[item_id,:].argsort()[-top_k:][::-1]
    topItems = [map_name[e] for e in topItems]
    return topItems

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

itemCorrMat = cosine_similarity(itemFeatures)

#before test_split it is mandatory to create dict mapping indexes of products in productsDF to labels or ids
#because truncatedSVD has rows coresponding to rows in productsDF, but after split there not the same
recommendations = top_k(6, 10, itemCorrMat, idxNameDict)
display(productsDF.loc[productsDF['product_id'].isin(recommendations)])

,product_id,product_name,category_path,price,user_rating
6,1007,Dead Space 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,3.995136
14,1015,Max Payne 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,2.696124
15,1016,Rayman Origins (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,4.659137
19,1020,Dead Space 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.00,4.774851
128,1129,Battlefield Bad Company 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,1.575231
160,1161,XCOM Enemy Unknown (PC),Gry i konsole;Gry komputerowe,18.90,4.929244
164,1165,Gdzie jest Nemo (PC),Gry i konsole;Gry komputerowe,19.99,4.316899
174,1175,Europa Universalis 3 (PC),Gry i konsole;Gry komputerowe,122.99,1.417595
265,1266,LA Noire (PC),Gry i konsole;Gry komputerowe,47.90,2.316601
315,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,249.00,4.287721
